# Original Code

In [31]:
import tkinter as tk
import random
import math
import time

class Bird:
    def __init__(self,name):
        self.x = random.randrange(100,900)
        self.y = random.randrange(100,900)
        self.angle = random.uniform(0.0,2.0*math.pi)
        self.name = name
        self.color = "blue"

    def draw(self,canvas):
        size = 10
        baseSize = 5
        point1 = [self.x + size*math.cos(self.angle), self.y + size*math.sin(self.angle)]
        point2 = [self.x - size*math.cos(self.angle) + baseSize*math.cos(self.angle+math.pi/2.0), \
                  self.y - size*math.sin(self.angle) + baseSize*math.sin(self.angle+math.pi/2.0) ]
        point3 = [self.x - size*math.cos(self.angle) - baseSize*math.cos(self.angle+math.pi/2.0), \
                  self.y - size*math.sin(self.angle) - baseSize*math.sin(self.angle+math.pi/2.0) ]
        canvas.create_polygon(point1+point2+point3,fill=self.color,tags=self.name)

    def move(self,canvas):
        distance = 5
        self.x += distance*math.cos(self.angle)
        self.y += distance*math.sin(self.angle)
        self.x = self.x%1000.0
        self.y = self.y%1000.0
        canvas.delete(self.name)
        self.draw(canvas)

    def distance(self,otherBird):
        return math.sqrt( (self.x-otherBird.x)*(self.x-otherBird.x) + \
                          (self.y-otherBird.y)*(self.y-otherBird.y) )

def initialise(window):
    window.resizable(False,False)
    canvas = tk.Canvas(window,width=1000,height=1000)
    canvas.pack()
    return canvas

def addBirds(canvas):
    birdList = []
    noOfBirds = 50
    for n in range(noOfBirds):
        bird = Bird("bird"+str(n))
        birdList.append(bird)
        bird.draw(canvas)
    return birdList

def moveBirds(canvas,listOfBirds):
    #find neighbours
    for bird in listOfBirds:
        nearbyBirds = []
        for otherBird in listOfBirds:
            if bird.distance(otherBird)<100 and (not bird.distance(otherBird)==0):
                nearbyBirds.append(otherBird)
        nearestBird = None
        if nearbyBirds:
            shortestDistance = 1000000.0
            for otherBird in nearbyBirds:
                d = bird.distance(otherBird)
                if d < shortestDistance:
                    shortestDistance = d
                    nearestBird = otherBird

        #move 1: move away from nearest
        if nearestBird is not None and bird.distance(nearestBird)<50 :
            if nearestBird.x-bird.x==0.0:
                angle = math.atan( (nearestBird.y-bird.y)/0.0001 )
            else:
                angle = math.atan( (nearestBird.y-bird.y)/(nearestBird.x-bird.x) )
            bird.angle -= angle/2.0

        #move 2: orient towards the neighbours
        averageAngle = 0.0
        if nearbyBirds:
            for nb in nearbyBirds:
                averageAngle += nb.angle
            averageAngle /= len(nearbyBirds)
            #bird.angle -= (averageAngle-bird.angle)/100.0
            bird.angle = averageAngle

        #move 3: move together
        if nearbyBirds:
            avX = 0.0
            avY = 0.0
            for nb in nearbyBirds:
                avX += nb.x
                avY += nb.y
            avX /= len(nearbyBirds)
            avY /= len(nearbyBirds)
            if avX-bird.x==0.0:
                angle = math.atan( (avY-bird.y)/0.0001 )
            else:
                angle = math.atan( (avY-bird.y)/(avX-bird.x) )
            bird.angle -= angle/20.0
            
    for bird in listOfBirds:
        bird.move(canvas)
    canvas.after(100,moveBirds,canvas,listOfBirds)
        
def main():
    window = tk.Tk()
    canvas = initialise(window)
    birdList = addBirds(canvas)
    moveBirds(canvas,birdList)
    window.mainloop()

main()


# My Code
## Boid Class

In [170]:
import numpy as np
import random
import tkinter as tk
import random
import math
import time

In [ ]:
class Bird:

    def __init__(self, name):
        self.x = random.randrange(100,900)
        self.y = random.randrange(100,900)
        self.angle = random.uniform(0.0,2.0*math.pi)
        self.name = name
        self.color = "blue"
      
    ### Helper functions ###
    
    def draw(self, canvas):
        size = 10
        baseSize = 5
        point1 = [self.x + size*math.cos(self.angle), self.y + size*math.sin(self.angle)]
        point2 = [self.x - size*math.cos(self.angle) + baseSize*math.cos(self.angle+math.pi/2.0), \
                  self.y - size*math.sin(self.angle) + baseSize*math.sin(self.angle+math.pi/2.0) ]
        point3 = [self.x - size*math.cos(self.angle) - baseSize*math.cos(self.angle+math.pi/2.0), \
                  self.y - size*math.sin(self.angle) - baseSize*math.sin(self.angle+math.pi/2.0) ]
        canvas.create_polygon(point1+point2+point3,fill=self.color,tags=self.name)
        
    def move(self,canvas):
        distance = 5
        self.x += distance*math.cos(self.angle)
        self.y += distance*math.sin(self.angle)
        self.x = self.x%1000.0
        self.y = self.y%1000.0
        canvas.delete(self.name)
        self.draw(canvas)

    def distance(self,otherAgent):
        return math.sqrt( (self.x-otherAgent.x)*(self.x-otherAgent.x) + \
                          (self.y-otherAgent.y)*(self.y-otherAgent.y) )

## The predator class

In [274]:
class Predator:
    def __init__(self, name):
        self.x = random.randrange(100,900)
        self.y = random.randrange(100,900)
        self.angle = random.uniform(0.0,2.0*math.pi)
        self.name = name
        self.color = "red"
      
    ### Helper functions ###
    
    def draw(self, canvas):
        size = 15
        baseSize = 8
        point1 = [self.x + size*math.cos(self.angle), self.y + size*math.sin(self.angle)]
        point2 = [self.x - size*math.cos(self.angle) + baseSize*math.cos(self.angle+math.pi/2.0), \
                  self.y - size*math.sin(self.angle) + baseSize*math.sin(self.angle+math.pi/2.0) ]
        point3 = [self.x - size*math.cos(self.angle) - baseSize*math.cos(self.angle+math.pi/2.0), \
                  self.y - size*math.sin(self.angle) - baseSize*math.sin(self.angle+math.pi/2.0) ]
        canvas.create_polygon(point1+point2+point3,fill=self.color,tags=self.name)
        
    def move(self,canvas):
        distance = 7 # slightly faster than the prey
        self.x += distance*math.cos(self.angle)
        self.y += distance*math.sin(self.angle)
        self.x = self.x%1000.0
        self.y = self.y%1000.0
        canvas.delete(self.name)
        self.draw(canvas)

    def distance(self,otherAgent):
        return math.sqrt( (self.x-otherAgent.x)*(self.x-otherAgent.x) + \
                          (self.y-otherAgent.y)*(self.y-otherAgent.y) )

## Rendering the simulation

### Boid movement 
Default Rules:
1. Separation (avoid crowding other flockmates)
2. Alignment (move at same angle as other flockmates)
3. Cohesion (move towards other flockmates)
Custom Rules:
4. Aversion (move away from the predator class)
5. Noise (add realism by adding some noise to the above)

In [334]:
def moveBirds(canvas,listOfBirds, perception_radius, minimum_radius, aversion_radius, predList, simulation_speed):
    
    #if listOfBirds is empty then return message
    if len(listOfBirds)<=0:
        print("no birds to move")
        return
    
    #find neighbours
    for bird in listOfBirds:
        
        ############### CUSTOM RULES 5 (noise) ##############
        
        # move 4 : aversion
        #check if any predators are within our aversion radius
        nearestPredator = None
        nearestPredatorDistance = 1000000
        for predator in predList:
            #if predator is within aversion radius and is lower than the nearestPredatorDistance...
            this_predator_distance = bird.distance(predator)
            if this_predator_distance<aversion_radius and this_predator_distance<nearestPredatorDistance:
                # update the neareast predator
                nearestPredator = predator
                nearestPredatorDistance = this_predator_distance
        #if nearestPredator is not null then avoid it
        if(nearestPredator!=None):
            #angle to face predator
            myradians = math.atan2(predator.y-bird.y, predator.x-bird.x)
            #inverting angle to face away from predator
            angleAway = math.radians((math.degrees(myradians) + 180)%360)
            #updating the angle
            bird.angle = angleAway 

        ################# END OF CUSTOM SEGMENT ##################
        
        #finding the birds wihtin the perception_radius
        nearbyBirds = []
        for otherBird in listOfBirds:
            if bird.distance(otherBird)<perception_radius and (not bird.distance(otherBird)==0):
                nearbyBirds.append(otherBird)
        #finding the closest bird of these birds
        nearestBird = None
        if nearbyBirds:
            shortestDistance = 1000000.0
            for otherBird in nearbyBirds:
                d = bird.distance(otherBird)
                if d < shortestDistance:
                    shortestDistance = d
                    nearestBird = otherBird

        #move 1: move away from nearest
        if nearestBird is not None and bird.distance(nearestBird)<minimum_radius :
            if nearestBird.x-bird.x==0.0:
                angle = math.atan( (nearestBird.y-bird.y)/0.0001 )
            else:
                angle = math.atan( (nearestBird.y-bird.y)/(nearestBird.x-bird.x) )
            bird.angle -= angle/2.0

#         #move 2: orient towards the neighbours
        
        ######## CUSTOM - MODIFIED THIS METHOD TO ONLY RUN IF WE ARE NOT AVOIDING PREDATOR AND TO EASE INTO ANGLE INSTEAD OF SNAPPING #############
        
        if(nearestPredator == None): 
            averageAngle = 0.0
            if nearbyBirds:
                for nb in nearbyBirds:
                    averageAngle += nb.angle
                averageAngle /= len(nearbyBirds)
                                
                ### if boids angle is within +/- certain range of degrees of average angle snap to it
                angle_range = 45 # n degree range
                angle_difference = (math.degrees(bird.angle) - math.degrees(averageAngle) + 180 + 360) % 360 - 180
                if (angle_difference <= angle_range and angle_difference>=-angle_range):
                    bird.angle = averageAngle

                #else increment the angle
                else:
                    #### rotating in the shortest direction
                    #if our current angle is lower than the angle we want to turn to
                    if(math.degrees(bird.angle)%360 < math.degrees(averageAngle)%360):
                        #if turning counterclockwise results in an angle < 180 this is the shortest route so do so
                        if(abs(math.degrees(bird.angle)%360 - math.degrees(averageAngle)%360) < 180):
                            bird.angle += 0.5
                        #else turning clockwise is the shortest route
                        else:
                            bird.angle -= 0.5
                    #else our angle is greater than the angle we want to turn to
                    else:
                        #if turning clockwise results in an angle < 180 this is the shortest route so do so
                        if(abs(math.degrees(bird.angle)%360 - math.degrees(averageAngle)%360) < 180):
                            bird.angle -= 0.5
                        #else turning counterclockwise is the shortest route
                        else:
                            bird.angle += 0.5

        ################# END OF CUSTOM SEGMENT ##################

        #move 3: move together
        if nearbyBirds:
            avX = 0.0
            avY = 0.0
            for nb in nearbyBirds:
                avX += nb.x
                avY += nb.y
            avX /= len(nearbyBirds)
            avY /= len(nearbyBirds)
            if avX-bird.x==0.0:
                angle = math.atan( (avY-bird.y)/0.0001 )
            else:
                angle = math.atan( (avY-bird.y)/(avX-bird.x) )
            bird.angle -= angle/20.0
            
        ############### CUSTOM RULES 5 (noise) ##############
               
        # move 5 : noise
        bird.angle*=random.randrange(85,115)/100 # adding some noise to the angle; between +/- 15%
        
        ################# END OF CUSTOM SEGMENT ##################
        
        
            
    for bird in listOfBirds:
        bird.move(canvas)
        bird.color="blue"
    canvas.after(simulation_speed,moveBirds,canvas,listOfBirds, perception_radius, minimum_radius, aversion_radius, predList, simulation_speed)

### Predator movement (100% custom)

In [357]:
def movePredators(canvas,predList, birdList, simulation_speed, birds_eaten):
    #for each predator
    for predator in predList:
        #finding the closest bird
        nearestBird = None
        shortestDistance = 1000000.0
        for bird in birdList:
            d = predator.distance(bird)
            if d < shortestDistance:
                shortestDistance = d
                nearestBird = bird
                
        #calculate the angle towards the nearest bird
        #if nearestBird is not null then attack it
        if(nearestBird!=None):
            
            #if this bird is close enough we will eat it
            if (predator.distance(nearestBird)<=10):
                birdList.remove(nearestBird)
                nearestBird.color="black"
                nearestBird.draw(canvas)
                birds_eaten +=1
                print(birds_eaten)
                continue
            
            
            
            #highlight the bird we are attacking
            nearestBird.color="magenta"
            
            #angle to face bird
            angleTowards = math.atan2(nearestBird.y-predator.y, nearestBird.x-predator.x)

            ### if predators angle is within +/- certain number of degrees
            angle_range =15 # n degree range
            angle_difference = (math.degrees(predator.angle) - math.degrees(angleTowards) + 180 + 360) % 360 - 180
            if (angle_difference <= angle_range and angle_difference>=-angle_range):
                predator.angle = angleTowards
            
            #else increment the angle
            else:
                #### rotating in the shortest direction
                #if our current angle is lower than the angle we want to turn to
                if(math.degrees(predator.angle)%360 < math.degrees(angleTowards)%360):
                    #if turning counterclockwise results in an angle < 180 this is the shortest route so do so
                    if(abs(math.degrees(predator.angle)%360 - math.degrees(angleTowards)%360) < 180):
                        predator.angle += 0.1
                    #else turning clockwise is the shortest route
                    else:
                        predator.angle -= 0.1
                #else our angle is greater than the angle we want to turn to
                else:
                    #if turning clockwise results in an angle < 180 this is the shortest route so do so
                    if(abs(math.degrees(predator.angle)%360 - math.degrees(angleTowards)%360) < 180):
                        predator.angle -= 0.1
                    #else turning counterclockwise is the shortest route
                    else:
                        predator.angle += 0.1
                        
                predator.angle*=random.randrange(85,115)/100 # adding some noise to the angle; between +/- 15%

        #if nearestBird is null then we have eaten all the birds so send message to stop early
        else: 
            print("eaten all birds")
            return birds_eaten
        
        
    
    #move each predator
    for predator in predList:
        predator.move(canvas)
    canvas.after(simulation_speed,movePredators,canvas,predList,birdList, simulation_speed, birds_eaten)

### Main Controller (custom changes)

'0'

In [359]:
####### CUSTOM CHANGED ADD BIRDS METHOD TO BE EASIER TO UNDERSTAND ######
def addBirds(canvas, num_boids):
    #creating the boids based on num_boids
    all_birds = []
    for i in range(num_boids):
        new_boid = Bird("bird_"+str(len(all_birds)))
        all_birds += [new_boid]
#         print(new_boid.name,": x="+str(new_boid.x)+", y="+str(new_boid.y))
    return all_birds
################# END OF CUSTOM SEGMENT ##################


########### CUSTOM METHOD FOR ADDING PREDATOR ############
def addPredators(canvas, num_predators):
    #creating the predators based on num_predators
    all_preds = []
    for i in range(num_predators):
        new_pred = Predator("predator_"+str(len(all_preds)))
        all_preds += [new_pred]
#         print(new_pred.name,": x="+str(new_pred.x)+", y="+str(new_pred.y))
    return all_preds
################# END OF CUSTOM SEGMENT ##################
        
def initialise(window):
    window.resizable(False,False)
    canvas = tk.Canvas(window,width=1000,height=1000, bg="black")
    canvas.pack()
    return canvas

def main(num_boids, perception_radius, minimum_radius, aversion_radius, num_preds, simulation_speed):
    birds_eaten = 0 
    window = tk.Tk()
    canvas = initialise(window)
    birdList = addBirds(canvas, num_boids) ### Custom now taking num_boids as a parameter to be tuned
    predList = addPredators(canvas, num_preds) ### Custom adding preadtors
    moveBirds(canvas,birdList, perception_radius, minimum_radius, aversion_radius, predList, simulation_speed) ### Custom now taking perception and minimum radius as parameters to be tuned
    birds_eaten = movePredators(canvas,predList, birdList, simulation_speed, 0) ### Custom moving predators
    window.mainloop()
    print("predator ate",birds_eaten,"boids (",str(round(birds_eaten/num_boids)),"%)")

    
birds_eaten = 0 
simulation_speed = 1

#number of boids
num_boids = 2

#number of predators
num_preds = 1

#radius in which the boids swarm behaviour kicks in
perception_radius = 100
#minimum radius between each boid
minimum_radius = 50
#radius in which aversion behaviour kicks in
aversion_radius = 200

    
main(num_boids, perception_radius, minimum_radius, aversion_radius, num_preds, simulation_speed)

1
2
no birds to move


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Users\Danie\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "D:\Users\Danie\Anaconda3\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "<ipython-input-357-8ac1eca6ba7e>", line 64, in movePredators
    test+="help"
UnboundLocalError: local variable 'test' referenced before assignment


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'